In [2]:
from pathlib import Path
import pandas as pd  # Added import for pandas
import geopandas as gpd
from shapely.geometry import LineString
import glob
import os

# Create a new folder if it does not exist
folder = Path("./02_geojson")
folder.mkdir(exist_ok=True)  # Corrected to boolean

def df_to_gdf(input_df):
    df = input_df.copy()

    def adjust_longitude(lon):
        # Ensure longitudes are within the range [-180, 180]
        return (lon + 180) % 360 - 180

    def create_linestring(row):
        dep_lon, dep_lat = adjust_longitude(row['dep_lon']), row['dep_lat']
        arr_lon, arr_lat = adjust_longitude(row['arr_lon']), row['arr_lat']

        # Handle the case where crossing the antimeridian is the shorter path
        if abs(dep_lon - arr_lon) > 180:
            if dep_lon > arr_lon:
                arr_lon += 360
            else:
                dep_lon += 360
        
        return LineString([(dep_lon, dep_lat), (arr_lon, arr_lat)])

    geometry = [create_linestring(row) for _, row in df.iterrows()]
    return gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

# Use glob to get all CSV files
files = glob.glob("./01_geocoding/*.csv")
#print(len(files), "ファイル")

for file in files:
    # Read the file
    filename = Path(file).stem  # Use Pathlib to get the file name without extension

    # Read data
    df = pd.read_csv(file)

        # Prepare data for pivot table
    df_intl = df[df["International/Domestic"] == "International"]

    # Create LineString from origin to destination coordinates
    gdf = df_to_gdf(df_intl)

    # Save as GeoJSON
    gdf.to_file(folder / f"fly-{filename}_linestring.geojson", driver="GeoJSON")  # Use Pathlib for consistency

print("終了しました")


終了しました
